In [ ]:
import pandas as pd
import numpy as np
import time
import copy

ex_pt = pd.read_excel("JSP_dataset.xlsx",sheet_name="Processing Time",index_col="Time")
ex_ms  = pd.read_excel("JSP_dataset.xlsx",sheet_name="Machines Sequence",index_col="order")

num_job = ex_pt.shape[1]
num_machine = ex_pt.shape[0]
num_gene = num_job*num_machine

#use map to avoid the type of number in df is not int (ex.string)
pt = [list(map(int,ex_pt.iloc[i]))for i in range(num_job)]
ms = [ list( map( int , ex_ms.iloc[i] ) ) for i in range(num_job)]

num_job = ex_pt.shape[1]
num_machine = ex_pt.shape[0]
num_gene = num_job*num_machine
population_size = int(input("enter population size (default:30) : " ) or 30)
crossover_rate = float(input("enter crossover rate (default:0.8) : ") or 0.8)
mutation_rate = float(input("enter mutation rate (default:0.2) : ") or 0.2)
mutation_select_rate = float(input("enter muation select rate (default:0.2) : ") or 0.2) #this rate is used to  a kind of mutation,to decide how many genes need to mutation
num_mutation = round(num_gene * mutation_select_rate)
iteration_num = int(input("enter the number of iteration times (default:2000) : ") or 2000)

#start to count the execution time
start_time = time.time()

#to set the initial population
Test_best = 9999999999
best_list,best_obj = [],[] #????????????
population_list = []

for i in range(population_size):
    JxM_rannum_list = list( np.random.permutation(num_gene) ) #because the type of permutation is "array"
    population_list.append(JxM_rannum_list)
    for j in range(num_gene):
        population_list[i][j] = population_list[i][j] % num_job # to make all number in list is 0-9

for i in range(iteration_num):
    Test_now = 9999999999
    parent_list = copy.deepcopy(population_list)
    child_list = copy.deepcopy(population_list)
    cross_seq = list(np.random.permutation(population_size)) #to decide which couple of gene crossover
    
    #crossover
    for j in range( int(population_size/2) ):
        crossover_probability = np.random.rand() # decide if need to crossover or not
        if( crossover_rate >= crossover_probability):
            parent_1 = population_list[ cross_seq[2*j] ]
            parent_2 = population_list[ cross_seq[2*j + 1]]
            #copy the parent first, then modify the gene later
            child_1 = parent_1[:]
            child_2 = parent_2[:]
            cut_point = list(np.random.choice(num_gene , 2 , replace = False))
            cut_point.sort() # to avoid the situation of cutpoint ex.(5,2)
            
            #modify the gene( crossover two chromosome between cutpoint)
            child_1[ cut_point[0] : cut_point[1] ] = parent_2[ cut_point[0] : cut_point[1] ]
            child_2[ cut_point[0] : cut_point[1] ] = parent_1[ cut_point[0] : cut_point[1] ]
            #put the child(chromosome) into the child_list
            child_list[ cross_seq[2*j] ] = child_1[:]
            child_list[ cross_seq[2*j +1] ] = child_2[:]
    #rapairment
    for j in range( population_size ):
        job_count = []
        larger,less = [],[]
        for i in range(num_job):
            if i in child_list[j]:
                job_count.append([child_list[j].count(i),child_list[j].index(i)]) #this is to store[times of each job appear , the first appear position]
            else:
                job_count.append([child_list[j].count(i),0])
            if job_count[i][0] > num_machine:
                larger.append(i)
            elif job_count[i][0] < num_machine:
                less.append(i)
                
        for k in range( len(larger) ):#append on each job which is larger
            change_job = larger[k]
            while job_count[change_job][0] > num_machine:
                for x in range(len(less)):
                    if job_count[ less[x] ][0] < num_machine:
                        child_list[j][job_count[less[x]][1]] = less[x]
                        job_count[change_job][1] = child_list[j].index(change_job)#to find the next appear,because the original is change to the less
                        job_count[change_job][0] = job_count[change_job][0]-1
                        job_count[ less[x] ][0] = job_count[ less[x] ][0] + 1
                    if job_count[change_job][0] == num_machine:
                        break

enter population size (default:30) : 
enter crossover rate (default:0.8) : 
enter mutation rate (default:0.2) : 
enter muation select rate (default:0.2) : 
enter the number of iteration times (default:2000) : 


In [3]:
pt

[[29, 78, 9, 36, 49, 11, 62, 56, 44, 21],
 [43, 90, 75, 11, 69, 28, 46, 46, 72, 30],
 [91, 85, 39, 74, 90, 10, 12, 89, 45, 33],
 [81, 95, 71, 99, 9, 52, 85, 98, 22, 43],
 [14, 6, 22, 61, 26, 69, 21, 49, 72, 53],
 [84, 2, 52, 95, 48, 72, 47, 65, 6, 25],
 [46, 37, 61, 13, 32, 21, 32, 89, 30, 55],
 [31, 86, 46, 74, 32, 88, 19, 48, 36, 79],
 [76, 69, 76, 51, 85, 11, 40, 89, 26, 74],
 [85, 13, 61, 7, 64, 76, 47, 52, 90, 45]]